# Data Formating for the Story
In this notebook we are going to focus on the formating of the data that will displayed on the Data Story Website : [tweet-awareness.eu](http://tweet-awareness.eu). As this website will display maps using Jave Script directly, we will have to handle the native data format of the data : `.json`

In [3]:
# We first define our local path to the Git Hub page repository : 
story_path = '../../../Data-Story/'

## 1. Displaying the twitter results : 
We are going to save the twitter results in `.json` : 
- The data normalized 